In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize

In [6]:
N = 100 #個人の数
I = 4 #選択候補の数
T = 4 

In [7]:
dist = np.array(list(list(map(lambda j:abs(i-j),range(I))) for i in range(I)))
stay = np.eye(I) # 滞在リンク
    
stayhome = np.zeros((I,I)) 
stayhome[0,0]=1

home = np.zeros((I,I))
home[:,0] = 1

時間構造化

In [10]:
def loglikelihood_drl(b):
    beta = b[2]
    #beta = 0
    LL = 0
    instant = np.exp(b[0]*dist + b[1]*home) #即時効用マトリックス
    print(instant)
    for n in range(N): # 個人ごと，timestepごと
        #V=log(vz)
        vz = np.ones((T+1,I))
        
        for t in range(T,0,-1): # 逆向き
            the = instant * (vz[t,:]**beta) #exp(v) * exp(bV)
            zi = the.sum(axis=1)
            vz[t-1,:] = (zi==0)*1 + (zi!=0)*zi
        
        knode = list(route.loc[n*T:(n+1)*T,'k']) # routeは経路情報ということ．個人nに対応するn*T~n+1*Tまでを取り出している
        anode = list(route.loc[n*T:(n+1)*T,'a'])
        
        for t in range(T):
            k = int(knode[t])
            a = int(anode[t])
            
            LL += np.log(instant[k,a]) + beta*np.log(vz[t+1,a]) - np.log(vz[t,k])
    return -LL

b = np.array((1, 1, 1))
print(loglikelihood_drl(b))

[[ 2.71828183  2.71828183  7.3890561  20.08553692]
 [ 7.3890561   1.          2.71828183  7.3890561 ]
 [20.08553692  2.71828183  1.          2.71828183]
 [54.59815003  7.3890561   2.71828183  1.        ]]


NameError: name 'route' is not defined

In [ ]:
def loglikelihood_grl(b):
    beta = b[2]
    #beta = 0
    LL = 0
    instant = np.exp(b[0]*dist + b[1]*home) #即時効用マトリックス
    
    for n in range(N):
        #V=log(vz)
        vz = np.ones((T+1,I))
        
        for t in range(T,0,-1): # 逆向き
            the = instant * (vz[t,:]**beta) #exp(v) * exp(bV)
            zi = the.sum(axis=1)
            vz[t-1,:] = (zi==0)*1 + (zi!=0)*zi
        
        knode = list(route.loc[n*T:(n+1)*T,'k'])
        anode = list(route.loc[n*T:(n+1)*T,'a'])
        
        for t in range(T):
            k = int(knode[t])
            a = int(anode[t])
            
            LL += np.log(instant[k,a]) + beta*np.log(vz[t+1,a]) - np.log(vz[t,k])
    return -LL